# Numerical Experiments with the Propagation Matrix

The agoal of this notebook is to further investigate the behaiour of the Propagation matrix. As right had side the Advection equation was used.
The first Idea was to consider the Propagation Matrix as a linear combination of the propagation matrices of the seperate steps. Then it would it was tried to investigate if it is possible to create a linear combination of these for which a given vector with  $u_i = 1$ for $i=n$ and $u_i = 0$ elswere  would lead to a positive u. This is equivalent to the collum n having only entries $\geq$ 0. 

This is not possible for the advection with periodic boundaries. If there is a feasible b for one special n then the same b would also be feasible for any other n. This cannot be true because this would enshure the general positifity.

If the advection doesn't have periodic boundaries it maybe is possible, but gets increaasingly more difficult when the n gets smaler.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from nodepy import rk
import cvxpy as cp


import numpy.linalg as linalg


#Diagonally Implicit methods:
BE = rk.loadRKM('BE').__num__()
SDIRK23 = rk.loadRKM('SDIRK23').__num__()
SDIRK34 = rk.loadRKM('SDIRK34').__num__()
SDIRK54 = rk.loadRKM('SDIRK54').__num__()
TR_BDF2 = rk.loadRKM('TR-BDF2').__num__()


be = rk.loadRKM('BE').__num__()

fe =rk.loadRKM('FE').__num__()
rk4 = rk.loadRKM('RK44').__num__()
rk4x2 = rk4*rk4
ssp2 = rk.loadRKM('SSP22').__num__()
ssp3 = rk.loadRKM('SSP33').__num__()
ssp104 = rk.loadRKM('SSP104').__num__()
merson4 = rk.loadRKM('Merson43').__num__()
bs5 = rk.loadRKM('BS5').__num__()

ck5 = rk.loadRKM('CK5').__num__()
dp5 = rk.loadRKM('DP5').__num__()
pd8 = rk.loadRKM('PD8').__num__()

trbdf = rk.loadRKM('TR-BDF2').__num__()
be = rk.loadRKM('BE').__num__()
irk2 = rk.loadRKM('LobattoIIIA2').__num__()



#Extrapolation method
ex2 = rk.extrap(2,'implicit euler').__num__()
ex3 = rk.extrap(3,'implicit euler').__num__()
ex4 = rk.extrap(4,'implicit euler').__num__()
ex5 = rk.extrap(5,'implicit euler').__num__()
ex6 = rk.extrap(6,'implicit euler').__num__()
ex8 = rk.extrap(8,'implicit euler').__num__()

In [ ]:
def generate_G(rkm,L,dt):
    b_orig = rkm.b
    s = len(rkm.b)
    G= ['?'] * s
    for i in range(s):
        b = np.zeros_like(b_orig)
        b[i] =1
        rkm.b = b
        (G_,X) = rkm.propagation_matrix(L,dt)
        G[i] = G_
    return G
    
    

In [ ]:
N = 50
A= (-np.diag(np.ones(N))+np.diag(np.ones(N-1),1))
G = generate_G(ex3,A,3)
(G_,X) = ex3.propagation_matrix(A,3)
plt.pcolor(G_)
plt.colorbar()

In [ ]:
plt.pcolor(G[0])
plt.colorbar()

In [ ]:
plt.pcolor(G[1])
plt.colorbar()

In [ ]:
plt.pcolor(G[2])
plt.colorbar()

In [ ]:
plt.pcolor(G[3])
plt.colorbar()

In [ ]:
plt.pcolor(G[4])
plt.colorbar()

In [ ]:
plt.pcolor(G[5])
plt.colorbar()

Investigating if negative values apear due to the cropping of the matrix exponential or if the neagtive valeus are caused by erors in the discretisation. 

The solution for the ODE $u' = Lu$ is $u(t) = exp(Mt) u_0$

Yhis cann be expressed with the Matrix exponential

$$exp(Mt) = \sum_{k=0}^{\infty} \frac{1}{k!}(Mt)^k=\sum_{k=0}^{\infty} \frac{1}{k!}M^kt^k$$

Now we want to investigate why there are negative values appearing in the numeric approximation $u^n$. 
At first we want to test weather negative values appear in a cropped version of the matrix exponential

In [ ]:
#set time to 1
t = 0.1

In [ ]:
def matrexp(A,power,t):
    exp_trunc = np.zeros_like(A)
    for k in range(power+1):
        exp_trunc += 1/np.math.factorial(k)*t**k*np.linalg.matrix_power(A,k)
    return exp_trunc


In [ ]:
print(matrexp(A,0,t))

In [ ]:
print(matrexp(A,1,t))

In [ ]:
print(matrexp(A,2,t))

In [ ]:
print(matrexp(A,3,t))

In [ ]:
print(matrexp(A,4,t))

In [ ]:
from ipywidgets import interact, IntSlider, FloatSlider




def plot_power(power=0):
    plt.pcolor(np.linalg.matrix_power(A,power),cmap=plt.get_cmap('bwr'),vmin=-4, vmax=4)
    plt.colorbar()
    

In [ ]:
interact(plot_power,power=IntSlider(min=0,max=20,value=1));

In [ ]:
def plot_exp_trunc(power=0,t=t):
    exp_trunc = matrexp(A,power,t)
    
    plt.pcolor(exp_trunc,cmap=plt.get_cmap('bwr'),vmin=-4, vmax=4)
    plt.colorbar()
    
interact(plot_exp_trunc,
         power=IntSlider(min=1,max=20,value=1),
         t=FloatSlider(min=0.,max=2.,step=0.01,value=0.));

## See how the cone of influence in the stagevalues look like for a simple upwinding

In [ ]:
def f(u):
    du = np.zeros_like(u)
    du[1:] = (u[1:]-u[:-1])
    du[0] = u[0]-u[-1]
    return -du*len(u)

u = np.zeros(20)
u[5] = 1

dt = 0.001

rkm = dp5
s = len(rkm)
p = len(u)
A = rkm.A
K = np.zeros([p,s])

#calculate a single step
for i in range(s):        #compute Stages

    u_prime = u.copy()
    for m in range(i):
        u_prime += dt*A[i,m]*K[:,m]
            
        K[:,i] = f(u_prime)
            

            

In [ ]:
V = np.sign(K)
plt.pcolor(V,cmap=plt.get_cmap('bwr'),vmin=-0.5, vmax=0.5)
plt.colorbar()

In [ ]:
plt.spy(K)

In [ ]:
print(rkm)